In [1]:
try:
    from ompl import base as ob
    from ompl import geometric as og
except ImportError:
    # if the ompl module is not in the PYTHONPATH assume it is installed in a
    # subdirectory of the parent directory called "py-bindings."
    from os.path import abspath, dirname, join
    import sys
    sys.path.insert(0, join(dirname(dirname(abspath(__file__))), 'py-bindings'))
    from ompl import base as ob
    from ompl import geometric as og

def isStateValid(state):
    # Some arbitrary condition on the state (note that thanks to
    # dynamic type checking we can just call getX() and do not need
    # to convert state to an SE2State.)
    return state.getX() < .6

def planWithSimpleSetup():
    # create an SE2 state space
    space = ob.SE2StateSpace()

    # set lower and upper bounds
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(-1)
    bounds.setHigh(1)
    space.setBounds(bounds)

    # create a simple setup object
    ss = og.SimpleSetup(space)
    ss.setStateValidityChecker(ob.StateValidityCheckerFn(isStateValid))

    start = ob.State(space)
    # we can pick a random start state...
    start.random()
    # ... or set specific values
    start().setX(.5)

    goal = ob.State(space)
    # we can pick a random goal state...
    goal.random()
    # ... or set specific values
    goal().setX(-.5)

    ss.setStartAndGoalStates(start, goal)

    # this will automatically choose a default planner with
    # default parameters
    solved = ss.solve(1.0)

    if solved:
        # try to shorten the path
        ss.simplifySolution()
        # print the simplified path
        print(ss.getSolutionPath())


def planTheHardWay():
    # create an SE2 state space
    space = ob.SE2StateSpace()
    # set lower and upper bounds
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(-1)
    bounds.setHigh(1)
    space.setBounds(bounds)
    # construct an instance of space information from this state space
    si = ob.SpaceInformation(space)
    # set state validity checking for this space
    si.setStateValidityChecker(ob.StateValidityCheckerFn(isStateValid))
    # create a random start state
    start = ob.State(space)
    start.random()
    # create a random goal state
    goal = ob.State(space)
    goal.random()
    # create a problem instance
    pdef = ob.ProblemDefinition(si)
    # set the start and goal states
    pdef.setStartAndGoalStates(start, goal)
    # create a planner for the defined space
    planner = og.RRTConnect(si)
    # set the problem we are trying to solve for the planner
    planner.setProblemDefinition(pdef)
    # perform setup steps for the planner
    planner.setup()
    # print the settings for this space
    print(si.settings())
    # print the problem settings
    print(pdef)
    # attempt to solve the problem within one second of planning time
    solved = planner.solve(1.0)

    if solved:
        # get the goal representation from the problem definition (not the same as the goal state)
        # and inquire about the found path
        path = pdef.getSolutionPath()
        print("Found solution:\n%s" % path)
    else:
        print("No solution found")

In [2]:

planWithSimpleSetup()
print("")
planTheHardWay()

Geometric path with 2 states
Compound state [
RealVectorState [0.5 0.344409]
SO2State [-1.86698]
]
Compound state [
RealVectorState [-0.5 0.638063]
SO2State [-2.35985]
]



Settings for the state space 'SE2CompoundSpace3'
  - state validity check resolution: 1%
  - valid segment count factor: 1
  - state space:
Compound state space 'SE2CompoundSpace3' of dimension 3 (locked) [
Real vector state space 'RealVectorSpace4' of dimension 2 with bounds: 
  - min: -1 -1 
  - max: 1 1 
 of weight 1
SO2 state space 'SO2Space5'
 of weight 0.5
]
Registered projections:
  - <default>
Projection of dimension 2
Cell sizes (computed defaults): [0.1 0.1]

Declared parameters:
longest_valid_segment_fraction = 0.01
projection.cellsize.0 = 0.1
projection.cellsize.1 = 0.1
projection.cellsize_factor = 0
valid_segment_count_factor = 1
Valid state sampler named uniform with parameters:
nr_attempts = 100

Start states:
Compound state [
RealVectorState [0.330976 -0.0747081]
SO2State [-2.65575]
]
Goal state, thr